In [ ]:
import urllib
import urllib.request
import os

def get_page(url):
    return urllib.request.urlopen(url).read().decode("iso-8859-1")

def is_int(txt):
    try:
        int(txt)
        return True
    except:
        return False

In [ ]:
def extract_calcudoku(url, outfname):
    html = get_page(url)
    str_in = html
    
    str_out = ''
    i=-1
    cells = []
    diff = ''
    for line in str_in.split('\n'):
        if 'class="grid2"' in line:
            i+=1
            continue
        if i==-1:
            continue
        if line.strip()=="</tr>":
            continue
        if 'Difficulty' in line:
            diff = line.split('Difficulty: ')[1].split('(')[0].strip()
            break
        if not 'class="math"' in line:
            continue

        while(len(cells) != i): cells.append([])

        cells[-1].append((line.split('class="')[1].split('"')[0],
                          line.split('<td class="math">')[1].split('<')[0]))
    nrows = len(cells)
    ncols = len(cells[0])

    wdt = 0
    for line in cells:
        for _,i in line:
            wdt = max(wdt, len(i))

    str_out += ('+' + '+'.join(['-'*wdt]*len(cells[0])) + '+') + '\n'

    for ii in range(nrows):
        pline = '|'
        for border, i in cells[ii]:
            pline += (i+" "*wdt)[:wdt] + ('|' if 'right' in border else " ")
        str_out += pline + '\n'

        pline = '+'
        for border, i in cells[ii]:
            pline += (("-" if 'bottom' in border else " ")*wdt) + '+' 
        str_out += pline + '\n'
    

    with open(outfname, 'w') as f:
        f.write('#source '+url+'\n')
        f.write('#difficulty: '+diff+'\n\n')
        f.write(str_out)

            
import time
nums = list(range(1,4599))

while(True):
    try:
        for i in nums:
            fname = '../calcudoku-collection/calcudoku-menneske.no-%d.txt'%i
            os.makedirs(os.path.dirname(fname), exist_ok=True)
            try:
                fin = open(fname).read()
            except:
                pass
            else:
                if len(fin) >= 100:
                    continue
                
            print(fname)
            url = 'http://www.menneske.no/calcudoku/9/eng/utskrift.html?number=%d'%i
            extract_calcudoku(url, fname)
        break
             
    except:
        print('timeout...')
        time.sleep(5)